In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

GRAPH_NAME = "infl_goods_serv"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [2]:
goods_serv_df = pd.read_csv(
    ut.get_repo_root() / "code" / "data" / "goods_services.csv")

goods_serv_df["Date"] = pd.to_datetime(goods_serv_df["Unnamed: 0"].astype(str), format="%Y%m")

with_index_df = goods_serv_df.set_index("Date").dropna()
clean_df = with_index_df.drop(columns=["Unnamed: 0"])



In [3]:
fig = go.Figure()

for col in clean_df.columns:
    fig.add_trace(
        go.Scatter(
            x=clean_df.index,
            y=clean_df[col],
            name=col,
            mode='lines',
        )
    )


# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
)

fig.update_layout(
    barmode='relative',
    title=dict(text='Inflation by Category'),
    xaxis_title='Date',
    yaxis_title='Change from Year Ago (%)',
)


# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

fig.show()
